In [1]:
# it seems the tcgplayer api is no longer available for new people, so I will have to scrape some data on my own

In [2]:
from selenium import webdriver
import time
import math
import re
import itertools
import pandas as pd
import selenium.webdriver.support.ui as ui

In [100]:
# this is how to obtain a title
def get_title(html_source, item_number):
    regex_title = re.compile('{}(.*){}'.format(re.escape('search-result__title">'), re.escape('</span><div class="inventory"')))
    html_title = re.findall('search-result__title">[^<]+</span><div class="inventory"',html_source)
    title = regex_title.findall(html_title[item_number])
    return(title)

In [101]:
def get_table_data(html, title):
    # the text we're interested in is tbody then the dates and price that follows
    regex_table = re.compile('{}(.*){}'.format(re.escape('tbody'), re.escape('tbody')))
    html_table_1 = regex_table.findall(html)
    dates = re.findall('\d{1,2}/\d{1,2} to \d{1,2}/\d{1,2}',html_table_1[0])
    prices = re.findall('\$\d+(?:\.\d+)?',html_table_1[0])
    regex_foil = re.compile('{}(.*){}'.format(re.escape('<div class="charts-title" data-v-f69e3427="">'), re.escape('</div>')))
    html_foil = re.findall('<div class="charts-title" data-v-f69e3427="">[^<]+</div>',html)
    foil = [regex_foil.findall(html_foil[i]) for i in range(len(html_foil))]
    foils = foil * len(dates)
    if len(dates) < len(prices):
        r = int(len(prices)/len(dates))
        dates = [item for item in dates for _ in range(r)] 
    titles = title * len(dates)
    return titles, foils, dates, prices

In [123]:
# url = 'https://www.tcgplayer.com/search/pokemon/sv-scarlet-and-violet-151?productLineName=pokemon&setName=sv-scarlet-and-violet-151&view=grid&page=1'
url = 'https://www.tcgplayer.com/search/pokemon/sv03-obsidian-flames?productLineName=pokemon&setName=sv03-obsidian-flames&view=grid&page=1'
driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver.get(url)
time.sleep(7)
html_source = driver.page_source



# how to find number of search results
regex = re.compile('{}(.*){}'.format(re.escape('search-result-count'), re.escape(' results')))
shortened_html = regex.findall(html_source)
num_results = re.findall('^.*>([0-9]+)$',shortened_html[0])
num_results = int(num_results[0])
num_pages = math.ceil(num_results/24)


htmls = []
titles = []
dates = []
prices = []
foils = []

# Sort A-Z so the products stay in order
sort_toggle = driver.find_element_by_class_name('tcg-input-select__trigger-toggle-button')
sort_toggle.click() 
time.sleep(3)
xpath = '/html/body/div[2]/div/div/section[2]/section/div/div[2]/div/div/div[2]/div/div/div/div/ul/li[3]/span'
a_to_z_sort = driver.find_element_by_xpath(xpath)
a_to_z_sort.click()
    
time.sleep(10)

for h in range(num_pages):
    # obtains the number of digits from the previous page so it can be removed properly from the end
    num_len = len(str(h))
    # go to the next page
    url = url[:-num_len] + str(h+1)
    driver.get(url)    

    time.sleep(12)
    item_titles = driver.find_elements_by_class_name('search-result__title')
    for i in range(len(item_titles)):
        html_source = driver.page_source
        title = get_title(html_source, i)
        
        item_titles = driver.find_elements_by_class_name('search-result__title')
#         print(h+1, i, len(item_titles))
        item_titles[i].click()
        time.sleep(12)
        html = driver.page_source
        htmls.append(html)
        
        title, foil, date, price = get_table_data(html, title)
#         if title[0] in titles:
#             print(title[0])
#         print(price[-1])
        titles.append(title)
        foils.append(foil)
        dates.append(date)
        prices.append(price)
        
        driver.back()
        time.sleep(15)


In [124]:
len(titles), len(dates), len(prices), len(foils)

(254, 254, 254, 254)

In [125]:
unique_titles = []
num = 0
# check to see that all titles are unique
for title in titles:
    if title[0] in unique_titles:
        print(title[0])
        print(num)
    else:
        unique_titles.append(title[0])
    num += 1
    

In [126]:
titles = list(itertools.chain(*titles))
dates = list(itertools.chain(*dates))
prices = list(itertools.chain(*prices))
foils = list(itertools.chain(*foils))

In [127]:
len(titles), len(dates), len(prices), len(foils)

(12930, 12930, 12930, 12930)

In [128]:
market_prices = pd.DataFrame([titles, foils, dates, prices]).T
market_prices = market_prices.rename(columns={0: 'Product', 1:'Foil', 2:'Date Range', 3: 'Market Price'})
market_prices

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,[Holofoil Market Price: ],7/18 to 7/20,$0.00
1,Absol ex - 135/197,[Holofoil Market Price: ],7/21 to 7/23,$0.00
2,Absol ex - 135/197,[Holofoil Market Price: ],7/24 to 7/26,$0.00
3,Absol ex - 135/197,[Holofoil Market Price: ],7/27 to 7/29,$0.00
4,Absol ex - 135/197,[Holofoil Market Price: ],7/30 to 8/1,$4.87
...,...,...,...,...
12925,Zigzagoon,[Reverse Holofoil Market Price: ],10/7 to 10/9,$0.06
12926,Zigzagoon,[ Market Price: ],10/10 to 10/12,$0.03
12927,Zigzagoon,[Reverse Holofoil Market Price: ],10/10 to 10/12,$0.06
12928,Zigzagoon,[ Market Price: ],10/13 to 10/15,$0.03


In [129]:
len(market_prices['Product'].unique())

254

In [130]:
# market_prices.to_csv('Pokemon_151.csv')
market_prices.to_csv('Obsidian_Flames.csv')